In [2]:
import re 
import emoji
import keras
import string
import numpy as np
import pandas as pd
import sklearn
from cleantext import clean
from keras import layers
from keras.models import Sequential
from matplotlib import pyplot as plt
from sklearn import (feature_extraction, linear_model, model_selection, preprocessing)
import textblob
from textblob import Word
import contractions
import nltk
from nltk.corpus import stopwords
from collections import Counter

train_data=pd.read_csv(r'C:\downloded dataset\twitter disater\processed_data\spell_corrected_text_tt_final.csv')
test_data=pd.read_csv(r'C:\downloded dataset\twitter disater\processed_data\test_spell_corrected_text_tt_final.csv')
print(len(train_data))
'''train_data.drop(columns=['id','keyword','location'],inplace=True)
test_data.drop(columns=['keyword','location'],inplace=True)'''

7613


"train_data.drop(columns=['id','keyword','location'],inplace=True)\ntest_data.drop(columns=['keyword','location'],inplace=True)"

In [4]:
#將縮寫還原
CHAT_WORDS_STR = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
chat_words_dict={}
chat_words_list=set()
for line in CHAT_WORDS_STR.split('\n'):
    if line!='':
        shortcut=line.split('=')[0]
        ori_words=line.split('=')[1]
        chat_words_list.add(shortcut)
        chat_words_dict[shortcut]=ori_words #建立縮寫字的字典
print(chat_words_dict)
def chat_words_conversion(text):
    new_text = []
    text=str(text)
    for word in text.split():
        if word.upper() in chat_words_dict.keys(): 
            text=text.replace(word,chat_words_dict[word.upper()])
    return text
#context = 'Guys! brb need to go. I will be AFK next couple of mins'
#print(chat_words_conversion(context))
train_data['text']=train_data['text'].apply(lambda text:chat_words_conversion(text))
test_data['text']=test_data['text'].apply(lambda text:chat_words_conversion(text))

{'AFAIK': 'As Far As I Know', 'AFK': 'Away From Keyboard', 'ASAP': 'As Soon As Possible', 'ATK': 'At The Keyboard', 'ATM': 'At The Moment', 'A3': 'Anytime, Anywhere, Anyplace', 'BAK': 'Back At Keyboard', 'BBL': 'Be Back Later', 'BBS': 'Be Back Soon', 'BFN': 'Bye For Now', 'B4N': 'Bye For Now', 'BRB': 'Be Right Back', 'BRT': 'Be Right There', 'BTW': 'By The Way', 'B4': 'Before', 'CU': 'See You', 'CUL8R': 'See You Later', 'CYA': 'See You', 'FAQ': 'Frequently Asked Questions', 'FC': 'Fingers Crossed', 'FWIW': "For What It's Worth", 'FYI': 'For Your Information', 'GAL': 'Get A Life', 'GG': 'Good Game', 'GN': 'Good Night', 'GMTA': 'Great Minds Think Alike', 'GR8': 'Great!', 'G9': 'Genius', 'IC': 'I See', 'ICQ': 'I Seek you (also a chat program)', 'ILU': 'ILU: I Love You', 'IMHO': 'In My Honest/Humble Opinion', 'IMO': 'In My Opinion', 'IOW': 'In Other Words', 'IRL': 'In Real Life', 'KISS': 'Keep It Simple, Stupid', 'LDR': 'Long Distance Relationship', 'LMAO': 'Laugh My A.. Off', 'LOL': 'Laug

In [ ]:
#處理縮寫字(contraction)，例如：I'm 

def convert_contraction(text):
    correct_word=[]
    for word in text.split():
        if word not in contractions.contractions_dict:
            correct_word.append(word)
        elif word in contractions.contractions_dict:
            correct_word.append(contractions.fix(word))
    return ' '.join(correct_word)
example = "I've worked so hard today. I'm going to run to home!"
print(convert_contraction(example))
#train_data['text']=train_data['text'].apply(lambda text:convert_contraction(text))
#test_data['text']=test_data['text'].apply(lambda text:convert_contraction(text))#remove stop words 

In [ ]:
#處理emoticons (-_-)
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}
def replace_emoticons(text):
    for i in EMOTICONS:
        text=re.sub(i,'_'.join(EMOTICONS[i].replace(',',' ').split()),text)
    return text
train_data['text']=train_data['text'].apply(lambda text:replace_emoticons(text))
test_data['text']=test_data['text'].apply(lambda text:replace_emoticons(text))

In [ ]:
#移除表情符號(emoji)
def remove_emoji(text):
    txt_emoji_list = emoji.distinct_emoji_list(text) #擷取emoji
    rx='['+ re.escape(''.join(txt_emoji_list)) +']' #用 re.escape 將emoji轉成正規表達式，在用[]包裝成集合

    if not txt_emoji_list:
        return text
    elif '©' not in txt_emoji_list:
        return re.sub(rx,'',text)+\
        ''.join([emoji.demojize(emo).replace(':','') for emo in txt_emoji_list])# demojize() 會出現:_:的形式
    elif '©' in txt_emoji_list:
        if len(txt_emoji_list)==1:
            return text.replace('©','')
        elif len(txt_emoji_list)!=1:
            text=text.replace('©','')
            return re.sub(rx,'',text)+\
            ''.join([emoji.demojize(emo).replace(':','') for emo in txt_emoji_list])
train_data['text']=train_data['text'].apply(lambda text:remove_emoji(text))#test_data 中無 emoji

In [ ]:
#移除html tag
def remove_html(text):
    html=re.sub(r'<.*?>','',text)
    text_no_whitespace=re.sub(r'\s{2,}',' ',html)
    return text_no_whitespace
'''example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""
print(remove_html(example))'''
train_data['text']=train_data['text'].apply(lambda text:remove_html(text))
test_data['text']=test_data['text'].apply(lambda text:remove_html(text))
'''
output = pd.DataFrame(train_data, columns=['text'])
output.to_csv(r'C:\downloded dataset\twitter disater\processed_data\spell_corrected_text_tt.csv', index=False)'''
output1=pd.DataFrame(test_data,columns=['text'])
output1.to_csv(r'C:\downloded dataset\twitter disater\processed_data\test_havnt_spell_corrected_text_tt.csv',index=False)


In [9]:
#移除標點符號
puts=string.punctuation #python中的標點符號
def remove_punctuatuon(text):
    return text.translate(str.maketrans('','',puts))
ex='ems1 ny emts petition 17 per hour ?繩minimum wage?穠 httpskepticalannoyedundecideduneasyorhesitanttco4oa6swlxmr ems paramedics ambulance'
print(remove_punctuatuon(ex))
train_data['text']= train_data["text"].apply(lambda text:remove_punctuatuon(text))
test_data['text']=test_data['text'].apply(lambda text:remove_punctuatuon(text))

ems1 ny emts petition 17 per hour 繩minimum wage穠 httpskepticalannoyedundecideduneasyorhesitanttco4oa6swlxmr ems paramedics ambulance


"output = pd.DataFrame(train_data, columns=['text'])\noutput.to_csv(r'C:\\downloded dataset\twitter disater\\processed_data\\spell_corrected_text_3.csv', index=False)\noutput1=pd.DataFrame(test_data,columns=['text'])\noutput1.to_csv(r'C:\\downloded dataset\twitter disater\\processed_data\test_spell_corrected_text_3.csv',index=False)"

In [10]:
#移除連結
def scrape_links(text):
    url = re.compile(r'http\w+')
    return url.sub(r'',text)
train_data['text']= train_data["text"].apply(lambda text:scrape_links(text))
test_data['text']=test_data['text'].apply(lambda text:scrape_links(text))

In [15]:
#移除特殊字元
s_chars = '¥₽ÏïŰŬĎŸæ₿₪ÚŇÀèÅ”ĜåŽÖéříÿý€ŝĤ₹áŜŮÂ₴ûÌÇšŘúüëÓ₫ŠčÎŤÆÒœ₩öËäøÍťìĈôàĥÝ¢ç“žðÙÊĉŭÈŒÐÉÔĵùÁů„âÄűĴóêĝÞîØòď฿ČÜþňÛ'
def remove_spec(text):
    text = re.sub(r'[\t\s\n\r\b\a]', ' ', text)
    text=re.sub(r'[{}]'.format(s_chars),'',text)
    text = re.sub(r'(\s[^iIaA]\s)', ' ', text)
    return text
train_data['text']= train_data["text"].apply(lambda text:remove_spec(text))
test_data['text']=test_data['text'].apply(lambda text:remove_spec(text))
example03 = train_data.iloc[2171,0]
#print(remove_spec(example03))


In [16]:
#統一大小寫
train_data['text']=train_data['text'].str.lower()
test_data['text']=test_data['text'].str.lower()

In [ ]:
#拼寫糾正
from tqdm import tnrange, tqdm_notebook,tqdm
from time import sleep

def correct_wd(text):
    correct_text=[]
    f=0
    zzz=tqdm_notebook(text.split())
    for i in zzz:
        f+=1
        i=str(i)
        i=Word(i)
        c=i.spellcheck()
        zzz.set_description('{}th iteration'.format(f))
        correct_text.append(c[0][0])
    return ' '.join(correct_text)
train_data['text']=train_data['text'].apply(lambda text:correct_wd(text))
test_data['text']=test_data['text'].apply(lambda text:correct_wd(text))

#context = 'Strawberyy is a beauity'
#print(correct_wd(context))
output = pd.DataFrame(train_data, columns=['text'])
output.to_csv(r'C:\downloded dataset\twitter disater\processed_data\spell_corrected_text_tt.csv', index=False)
output1=pd.DataFrame(test_data,columns=['text'])
output1.to_csv(r'C:\downloded dataset\twitter disater\processed_data\test_spell_corrected_text_tt.csv',index=False)

In [29]:
#處理數字
train_data['text']=train_data['text'].str.replace('\d+','',regex=True)
test_data['text']=test_data['text'].str.replace('\d+','',regex=True)


In [30]:
pattern=re.compile(r'(.)\1{3,}')
def reduce_length(text):
    return pattern.sub(r'\1',text)
train_data['text']=train_data['text'].apply(lambda text:reduce_length(text))
test_data['text']=test_data['text'].apply(lambda text:reduce_length(text))


In [31]:
# 去除停用詞
stopwords_list=stopwords.words('english')
print(stopwords_list)
def remove_stopwords(text):
    correct_words=[]
    for i in text.split():
        if i in stopwords_list:
            continue
        else:
            correct_words.append(i)
    return ' '.join(correct_words)
train_data['text']=train_data['text'].apply(lambda text:remove_stopwords(text))
test_data['text']=test_data['text'].apply(lambda text:remove_stopwords(text))
print(train_data['text'].head())

0         deeds reason earthquake may allah forgive us
1                 forest fire near la range ask canada
2    residents asked shelter place notified officer...
3    people receive wildfires evacuation orders cal...
4    got sent photo ruby alaska smoke wildfires hou...
Name: text, dtype: object


In [32]:
#去掉高低頻詞
DISASTER_WORDS = ['fire', 'kill', 'bomb', 'disaster', 'crash', 'flood',
                  'suicide', 'police', 'attack', 'storm', 'emergency', 'burn']

NON_DISASTER_WORDS = ['like', 'new', 'body', 'love', 'good']
cnt=Counter()
for text in train_data["text"]:
    for word in text.split():
        cnt[word]+=1
most=cnt.most_common(10)
least=cnt.most_common()[:-10:-1]

mot_and_lets=[]
for i in most:
    mot_and_lets.append(i[0])
for i in least:
    mot_and_lets.append(i[0])
print(mot_and_lets)
for word in DISASTER_WORDS + NON_DISASTER_WORDS:
    if word in mot_and_lets:
        mot_and_lets.pop(mot_and_lets.index(word))
def remove_freq(text):
        return " ".join([word for word in text.split() if word not in mot_and_lets])
train_data['text']=train_data['text'].apply(lambda text:remove_freq(text))
test_data['text']=test_data['text'].apply(lambda text:remove_freq(text))
output = pd.DataFrame(train_data, columns=['text'])
output.to_csv(r'C:\downloded dataset\twitter disater\processed_data\spell_corrected_text_tt_final.csv', index=False)
output1=pd.DataFrame(test_data,columns=['text'])
output1.to_csv(r'C:\downloded dataset\twitter disater\processed_data\test_spell_corrected_text_tt_final.csv',index=False)

['like', 'amp', 'fire', 'get', 'via', 'new', 'news', 'dont', 'people', 'us', 'symptoms', 'developing', 'bathandnortheastsomerset', 'overtaking', 'fatherofthree', 'explosivespacked', 'explodingkittens\x89', 'gameofkittens', 'exploding']


In [ ]:
#